
- File name: Tarea3_Equipo3_fem.ipynb
- Author: Maria Paula Rey
- Date last modified: 22/10/2021
- Python Version: 3.8


# PUNTO 4

Este codigo obtiene las matrices de carga elementales y globales para funciones f arbitrarias y finalmente resuelve el sistema de ecuaciones resultante. 

Primero debemos importar las librerías necesarias.

In [1]:
from sympy import zeros
import numpy as np
import matplotlib.pyplot as plt
from sympy import *
from sympy import plotting
import sympy as sym 

%matplotlib notebook
init_printing()

Ahora definimos la función $f(x)$ que va a ser la carga de la ecuación diferencial:

$$\frac{d^2u}{dx^2} = f(x) $$

Esta ecuación puede ser modificada.

con condiciones de frontera $u(0)=u(1)=0$



In [2]:
def f(x):
    return 3#x*(1-x)

## Hallando la matriz de carga $K^{(e)}$

Para esto se utilizan las funciones base $\phi_0$, $\phi_1$ y $\phi_2$, cada una respectiva a cada nodo del elemento cuadrático. 

In [6]:
# Creando las variables para los cálculos
x,r,h,x0 = symbols("x r h x0")

phi_0 = r*(r-1)/2
phi_1 = -(r+1)*(r-1)
phi_2 = r*(r+1)/2

phi_s = Matrix([phi_0, phi_1, phi_2])
xs = transpose(Matrix([x0, x0+h/2, x0+h]))

for phi in phi_s:
    display(expand(phi))

Ahora procedemeos a hallar $X(r)$ y a derivar respecto a $r$ para hallar el jacobiano $J$.

In [7]:
X = xs*phi_s
X = simplify(X)
display(X)
J = diff(X,r) # J = dx/dr
J = J[0]
display(J)

⎡h⋅r   h     ⎤
⎢─── + ─ + x₀⎥
⎣ 2    2     ⎦

Dado que el elemento $k_{i,j}$ de la matriz local es $\int\limits_\Omega \phi_i' \phi_j' \mathrm{d}\Omega\,$, se calculan las derivadas de las funciones base.

In [8]:
dphi_s = Matrix([simplify(J**-1*diff(phi_s[i],r)) for i in range(3)])
dphi_sT = transpose(dphi_s)
dphi_sT

⎡2⋅r - 1  -4⋅r   2⋅r + 1⎤
⎢───────  ─────  ───────⎥
⎣   h       h       h   ⎦

Finalmente se puene hallar la matriz $K^{(e)}$ elemental/de cada elemento:

In [9]:
A = zeros(3,3)
A = integrate(dphi_s*dphi_sT*J, (r,-1,1))
MatMul(gcd(tuple(A)),(A/gcd(tuple(A))), evaluate=False)

  ⎡7/3   -8/3  1/3 ⎤
1 ⎢                ⎥
─⋅⎢-8/3  16/3  -8/3⎥
h ⎢                ⎥
  ⎣1/3   -8/3  7/3 ⎦

## Hallando el vector de carga local

Para hallar el vector de carga local $c^{(e)}$, se necesitan saber primero los números de elementos a usar para así poder evaluar $f(x)$ en cada nodo:

In [10]:
n_elem = 5

Después se calcula el número de nodos y su espaciamiento

In [11]:
n_node = 2*n_elem+1
dx = 1/(2*n_elem)

Finalmente calculamos $c^{(e)}$ elemental/para cada elemento

In [12]:
#Pesos y puntos de Gauss

#xi = [-np.sqrt(3/7-2/7*np.sqrt(1.2)), np.sqrt(3/7-2/7*np.sqrt(1.2)),
#      -np.sqrt(3/7+2/7*np.sqrt(1.2)), np.sqrt(3/7+2/7*np.sqrt(1.2))]
xi = np.array([-0.774597, 0, 0.774597])

#wi = [(18+np.sqrt(30))/36,(18+np.sqrt(30))/36,
#      (18-np.sqrt(30))/36,(18-np.sqrt(30))/36]
wi = np.array([0.555556, 0.888889, 0.555556])

c_loc = np.zeros((3,n_elem))                                   # B local
 
## Calcular c_loc popr medio de la cuadratura de gauss
detJ = N(J.subs({'h':2*dx}))                                   # Reemplazar valores ya conocidos
X = X.subs({'h':2*dx})

for i in range(n_elem):                                        # Recorrer cada elemento
    for j in range(3):                                         # Recorrer cada phi
        sum_ = 0
        for k,rk in enumerate(xi):                             # Recorrer cada punto de la cuadratura
            aux = np.array(X.subs({'x0':2*i*dx,'r':rk})[0],dtype='float64')
            sum_ += phi_s[j].subs({'r':rk})*wi[k]*f(aux)
        c_loc[j,i] = detJ*sum_
print(c_loc)
print('Cada columna representa el vector de carga de cada elemento.')

[[0.10000017 0.10000017 0.10000017 0.10000017 0.10000017]
 [0.39999997 0.39999997 0.39999997 0.39999997 0.39999997]
 [0.10000017 0.10000017 0.10000017 0.10000017 0.10000017]]
Cada columna representa el vector de carga de cada elemento.


Ya conocemos el espaciamiento de cada nodo, entonces podemos determinar el valor numérico de la matriz $K^{(e)}$:

In [13]:
K_loc = -np.array(N(A.subs({'h':2*dx})), dtype='float64')
K_loc

array([[-11.66666667,  13.33333333,  -1.66666667],
       [ 13.33333333, -26.66666667,  13.33333333],
       [ -1.66666667,  13.33333333, -11.66666667]])

Finalmente se crean dos funciones que ensamblan las matriz global $C^G$ siguiendo la numeración local de cada nodo:

In [14]:
# Funciones para ensamblar C_G
def reordenar_vector(v,num_Local=[0,2,1]):
    return v[num_Local]

def ensamblar_C_global(c_local, Nx, num_Loc=[0,1,2]):
    
    C_g = np.zeros((Nx,1))
    
    # Nos interesa el numero de elementos y el numero de nodos por elemento
    n, nElm = c_local.shape
    for i in range(nElm):
        
        c = reordenar_vector(c_local[:,i],num_Loc)
        aux = C_g[i*(n-1):i*(n-1)+n] + np.reshape(c,(n,1))
        C_g[i*(n-1):i*(n-1)+n] = aux
    return C_g

Y otras dos funciones para ensamblar la matriz global $K^G$ siguiendo la numeración local de cada nodo:

In [15]:

# Algoritmos para ensamblar K_G
def reordenar_matriz(m, num_Local=[0, 2, 1]):
  return m[num_Local][:,num_Local]

def ensamblar_K_global(k_Loc, Nx, num_Loc=[0,1,2]):
    # Matriz K global
    K_g = np.zeros((Nx,Nx))
    
    # Como es una matriz cuadrada cogemos cualquier dimension
    n = k_Loc.shape[1]
    
    m = reordenar_matriz(k_Loc, num_Loc)
    
    for i in range(0, Nx-1,n-1):
        K_g[i:i+n,i:i+n] += m
    return K_g

Entonces la matriz de carga global $K^G$ es:

In [16]:
K_G = ensamblar_K_global(K_loc, n_node)#, [0,1,2])
print('K_G =',K_G)

K_G = [[-11.66666667  13.33333333  -1.66666667   0.           0.
    0.           0.           0.           0.           0.
    0.        ]
 [ 13.33333333 -26.66666667  13.33333333   0.           0.
    0.           0.           0.           0.           0.
    0.        ]
 [ -1.66666667  13.33333333 -23.33333333  13.33333333  -1.66666667
    0.           0.           0.           0.           0.
    0.        ]
 [  0.           0.          13.33333333 -26.66666667  13.33333333
    0.           0.           0.           0.           0.
    0.        ]
 [  0.           0.          -1.66666667  13.33333333 -23.33333333
   13.33333333  -1.66666667   0.           0.           0.
    0.        ]
 [  0.           0.           0.           0.          13.33333333
  -26.66666667  13.33333333   0.           0.           0.
    0.        ]
 [  0.           0.           0.           0.          -1.66666667
   13.33333333 -23.33333333  13.33333333  -1.66666667   0.
    0.        ]
 [  0.          

Y el vector de carga global $C^G$ es:

In [17]:
C_G = ensamblar_C_global(c_loc, n_node)#, [0,1,2])
print('C_G =',C_G)

C_G = [[0.10000017]
 [0.39999997]
 [0.20000033]
 [0.39999997]
 [0.20000033]
 [0.39999997]
 [0.20000033]
 [0.39999997]
 [0.20000033]
 [0.39999997]
 [0.10000017]]


## Resolviendo el sistema de ecuaciones

In [18]:
#WithScipy
from scipy import linalg
u = np.linalg.solve(K_G[1:-1,1:-1], C_G[1:-1])
print(u)

[[-0.13500006]
 [-0.24000012]
 [-0.31500015]
 [-0.36000018]
 [-0.37500018]
 [-0.36000018]
 [-0.31500015]
 [-0.24000012]
 [-0.13500006]]


In [20]:
#With sympy
from sympy import *
M = Matrix(K_G[1:-1,1:-1])
syms=symbols('u:%d'%M.shape[1])
M = M.col_insert(M.shape[1], Matrix(C_G[1:-1]))
display(M)
display(linsolve(M,syms))

⎡-26.6666666666667  13.3333333333333          0.0                0.0          
⎢                                                                             
⎢13.3333333333333   -23.3333333333333  13.3333333333333   -1.66666666666667   
⎢                                                                             
⎢       0.0         13.3333333333333   -26.6666666666667  13.3333333333333    
⎢                                                                             
⎢       0.0         -1.66666666666667  13.3333333333333   -23.3333333333333  1
⎢                                                                             
⎢       0.0                0.0                0.0         13.3333333333333   -
⎢                                                                             
⎢       0.0                0.0                0.0         -1.66666666666667  1
⎢                                                                             
⎢       0.0                0.0                0.0   

In [ ]:
M=Matrix(K_G[1:-1,1:-1])
B=Matrix(C_G[1:-1])
display(M)

⎡-26.6666666666667  13.3333333333333          0.0                0.0          
⎢                                                                             
⎢13.3333333333333   -23.3333333333333  13.3333333333333   -1.66666666666667   
⎢                                                                             
⎢       0.0         13.3333333333333   -26.6666666666667  13.3333333333333    
⎢                                                                             
⎢       0.0         -1.66666666666667  13.3333333333333   -23.3333333333333  1
⎢                                                                             
⎢       0.0                0.0                0.0         13.3333333333333   -
⎢                                                                             
⎢       0.0                0.0                0.0         -1.66666666666667  1
⎢                                                                             
⎢       0.0                0.0                0.0   

In [ ]:
M.shape[1]

In [ ]:
K_G[1:-1,1:-1]